In [1]:
# default_exp models.esrnn.esrnn

In [2]:
#hide
%load_ext autoreload
%autoreload 2

# ESRNN model

> API details.

#TODO: validation loader

In [3]:
#export
import os
import time

import numpy as np
import pandas as pd

import torch as t
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

from pathlib import Path
from copy import deepcopy

from nixtla.models.esrnn.utils.esrnn import _ESRNN
from nixtla.models.esrnn.utils.losses import SmylLoss, PinballLoss
from nixtla.models.esrnn.utils.data import Iterator

In [4]:
#export
class ESRNN(object):
    """ Exponential Smoothing Recurrent Neural Network

    Pytorch Implementation of the M4 time series forecasting competition winner.
    Proposed by Smyl. The model uses a hybrid approach of Machine Learning and
    statistical methods by combining recurrent neural networks to model a common
    trend with shared parameters across series, and multiplicative Holt-Winter
    exponential smoothing.

    Parameters
    ----------
    max_epochs: int
        maximum number of complete passes to train data during fit
    freq_of_test: int
        period for the diagnostic evaluation of the model.
    learning_rate: float
        size of the stochastic gradient descent steps
    lr_scheduler_step_size: int
        this step_size is the period for each learning rate decay
    per_series_lr_multip: float
        multiplier for per-series parameters smoothing and initial
        seasonalities learning rate (default 1.0)
    gradient_eps: float
        term added to the Adam optimizer denominator to improve
        numerical stability (default: 1e-8)
    gradient_clipping_threshold: float
        max norm of gradient vector, with all parameters treated
        as a single vector
    rnn_weight_decay: float
        parameter to control classic L2/Tikhonov regularization
        of the rnn parameters
    noise_std: float
        standard deviation of white noise added to input during
        fit to avoid the model from memorizing the train data
    level_variability_penalty: float
        this parameter controls the strength of the penalization
        to the wigglines of the level vector, induces smoothness
        in the output
    testing_percentile: float
        This value is only for diagnostic evaluation.
        In case of percentile predictions this parameter controls
        for the value predicted, when forecasting point value,
        the forecast is the median, so percentile=50.
    training_percentile: float
        To reduce the model's tendency to over estimate, the
        training_percentile can be set to fit a smaller value
        through the Pinball Loss.
    seasonality: int list
        list of seasonalities of the time series
        Hourly [24, 168], Daily [7], Weekly [52], Monthly [12],
        Quarterly [4], Yearly [].
    input_size: int
        input size of the recurrent neural network, usually a
        multiple of seasonality
    output_size: int
        output_size or forecast horizon of the recurrent neural
        network, usually multiple of seasonality
    random_seed: int
        random_seed for pseudo random pytorch initializer and
        numpy random generator
    min_inp_seq_length: int
        description
    cell_type: str
        Type of RNN cell, available GRU, LSTM, RNN, ResidualLSTM.
    state_hsize: int
        dimension of hidden state of the recurrent neural network
    dilations: int list
        each list represents one chunk of Dilated LSTMS, connected in
        standard ResNet fashion
    add_nl_layer: bool
        whether to insert a tanh() layer between the RNN stack and the
        linear adaptor (output) layers
    device: str
        pytorch device either 'cpu' or 'cuda'
    Notes
    -----
    **References:**
    `M4 Competition Conclusions
    <https://rpubs.com/fotpetr/m4competition>`__
    `Original Dynet Implementation of ESRNN
    <https://github.com/M4Competition/M4-methods/tree/master/118%20-%20slaweks17>`__
    """
    def __init__(self,
                 input_size=4,
                 output_size=8,
                 max_epochs=15,
                 freq_of_test=-1,
                 learning_rate=1e-3,
                 lr_scheduler_step_size=9,
                 lr_decay=0.9,
                 per_series_lr_multip=1.0,
                 gradient_eps=1e-8,
                 gradient_clipping_threshold=20,
                 rnn_weight_decay=0,
                 noise_std=0.001,
                 level_variability_penalty=80,
                 testing_percentile=50,
                 training_percentile=50,
                 cell_type='LSTM',
                 state_hsize=40,
                 dilations=[[1, 2], [4, 8]],
                 add_nl_layer=False,
                 seasonality=[4],
                 random_seed=1,
                 device='cpu', root_dir='./'):
        super(ESRNN, self).__init__()

        self.input_size = input_size
        self.output_size = output_size

        self.cell_type = cell_type
        self.state_hsize = state_hsize
        self.dilations = dilations
        self.add_nl_layer = add_nl_layer
        self.seasonality = seasonality

        self.max_epochs = max_epochs
        self.learning_rate = learning_rate
        self.lr_scheduler_step_size = lr_scheduler_step_size
        self.lr_decay = lr_decay
        self.per_series_lr_multip = per_series_lr_multip
        self.gradient_eps = gradient_eps
        self.gradient_clipping_threshold = gradient_clipping_threshold
        self.freq_of_test = freq_of_test

        self.rnn_weight_decay = rnn_weight_decay
        self.noise_std = noise_std
        self.level_variability_penalty = level_variability_penalty
        self.testing_percentile = testing_percentile
        self.training_percentile = training_percentile

        self.random_seed = random_seed
        self.device = device
        self.root_dir = root_dir
        self._fitted = False

    def to_tensor(self, x: np.ndarray, dtype = t.float32) -> t.Tensor:
        tensor = t.as_tensor(x, dtype=dtype).to(self.device)
        return tensor

    def __loss_fn(self, loss_name: str):
        def loss(x, freq, forecast, target, mask):
            if loss_name == 'SMYL':
                return 
            if loss_name == 'MAPE':
                return MAPELoss(y=target, y_hat=forecast, mask=mask)
            elif loss_name == 'MASE':
                return MASELoss(y=target, y_hat=forecast, y_insample=x, seasonality=freq, mask=mask)
            elif loss_name == 'SMAPE':
                return SMAPELoss(y=target, y_hat=forecast, mask=mask)
            elif loss_name == 'MSE':
                return MSELoss(y=target, y_hat=forecast, mask=mask)
            elif loss_name == 'MAE':
                return MAELoss(y=target, y_hat=forecast, mask=mask)
            else:
                raise Exception(f'Unknown loss function: {loss_name}')
        return loss
       
    # def evaluate_performance(self, ts_loader, validation_loss_fn):
    #     """
    #     Auxiliary function, evaluate ESRNN model for training
    #     procedure supervision.

    #     Parameters
    #     ----------
    #     dataloader: pytorch dataloader
    #     criterion: pytorch test criterion

    #     Returns
    #     -------
    #     model_loss: float
    #         loss for train supervision purpose.
    #     """
    #     #TODO: FALTA
    #     with t.no_grad():
    #         model_loss = 0.0
    #         for batch in iter(ts_loader):
    #             windows_y, windows_y_hat, _ = self.esrnn(batch)
    #             loss = validation_loss_fn(windows_y, windows_y_hat)
    #             model_loss += loss.data.cpu().numpy()
    #         model_loss /= dataloader.n_batches
    #     return model_loss

    def fit(self, train_ts_loader, val_ts_loader=None, max_epochs=None, verbose=True, eval_epochs=1):
        """
        Fit ESRNN model.

        Parameters
        ----------
        Returns
        -------
        self : returns an instance of self.
        """

        # Random Seeds (model initialization)
        t.manual_seed(self.random_seed)
        np.random.seed(self.random_seed)

        # Exogenous variables
        self.n_x, self.n_s = train_ts_loader.get_n_variables()
        self.frequency = train_ts_loader.get_frequency()
        print("Infered frequency: {}".format(self.frequency))

        # Initialize model
        self.n_series = train_ts_loader.get_n_series()
        self.instantiate_esrnn()

        # Train model
        self._fitted = True

        # dataloader, max_epochs, shuffle, verbose
        if verbose: print(15*'='+' Training ESRNN  ' + 15*'=' + '\n')

        # Optimizers
        self.es_optimizer = optim.Adam(params=self.esrnn.es.parameters(),
                                        lr=self.learning_rate*self.per_series_lr_multip,
                                        betas=(0.9, 0.999), eps=self.gradient_eps)

        self.es_scheduler = StepLR(optimizer=self.es_optimizer,
                                    step_size=self.lr_scheduler_step_size,
                                    gamma=0.9)

        self.rnn_optimizer = optim.Adam(params=self.esrnn.rnn.parameters(),
                                        lr=self.learning_rate,
                                        betas=(0.9, 0.999), eps=self.gradient_eps,
                                        weight_decay=self.rnn_weight_decay)

        self.rnn_scheduler = StepLR(optimizer=self.rnn_optimizer,
                                    step_size=self.lr_scheduler_step_size,
                                    gamma=self.lr_decay)

        # Loss Functions
        #TODO: cambiar por similar a Nbeats
        train_tau = self.training_percentile / 100
        train_loss = SmylLoss(tau=train_tau,
                              level_variability_penalty=self.level_variability_penalty)

        eval_tau = self.testing_percentile / 100
        eval_loss = PinballLoss(tau=eval_tau)

        # Overwrite n_iterations and train datasets
        if max_epochs is None:
            max_epochs = self.max_epochs

        start = time.time()
        self.trajectories = {'epoch':[],'train_loss':[], 'val_loss':[]}

        # for epoch in range(max_epochs):
        #     self.esrnn.train()
        #     start = time.time()

        #     losses = []
        # Training Loop
        for epoch in range(max_epochs):
            losses = []
            for batch in iter(train_ts_loader):
                self.esrnn.train()
                self.es_optimizer.zero_grad()
                self.rnn_optimizer.zero_grad()
                
                insample_y  = self.to_tensor(x=batch['insample_y'])
                s_matrix    = self.to_tensor(x=batch['s_matrix'])
                idxs        = self.to_tensor(x=batch['idxs'], dtype=t.long)

                windows_y, windows_y_hat, levels = self.esrnn(insample_y=insample_y, s_matrix=s_matrix, idxs=idxs)
                
                # Pinball loss on normalized values
                training_loss = train_loss(windows_y, windows_y_hat, levels)
                training_loss.backward()
                losses.append(training_loss.cpu().data.numpy())
                
                t.nn.utils.clip_grad_norm_(self.esrnn.rnn.parameters(),
                                            self.gradient_clipping_threshold)
                t.nn.utils.clip_grad_norm_(self.esrnn.es.parameters(),
                                            self.gradient_clipping_threshold)
                self.rnn_optimizer.step()
                self.es_optimizer.step()

                # Decay learning rate
                self.es_scheduler.step()
                self.rnn_scheduler.step()

            # Evaluation
            if (epoch % eval_epochs == 0):
                display_string = 'Epoch: {}, Time: {:03.3f}, Insample loss: {:.5f}'.format(epoch,
                                                                                time.time()-start,
                                                                                np.mean(losses))
                self.trajectories['epoch'].append(epoch)
                self.trajectories['train_loss'].append(training_loss.cpu().data.numpy())

                if val_ts_loader is not None:
                    loss = self.evaluate_performance(ts_loader=val_ts_loader, 
                                                        validation_loss_fn=eval_loss)
                    display_string += ", Outsample loss: {:.5f}".format(loss)
                    self.trajectories['val_loss'].append(loss)
                
                print(display_string)

                self.esrnn.train()
                train_ts_loader.train()

    def instantiate_esrnn(self):
        """Auxiliary function used at beginning of train to instantiate ESRNN"""
        self.esrnn = _ESRNN(n_series=self.n_series, input_size=self.input_size, output_size=self.output_size,
                            n_s=self.n_s, seasonality=self.seasonality,
                            noise_std=self.noise_std, cell_type=self.cell_type,
                            dilations=self.dilations, state_hsize=self.state_hsize,
                            add_nl_layer=self.add_nl_layer, device=self.device).to(self.device)


    def predict(self, ts_loader, X_test=None):
        assert self._fitted, "Model not fitted yet"
        self.esrnn.eval()
        ts_loader.eval()
        frequency = ts_loader.get_frequency()

        # Build forecasts
        unique_ids = ts_loader.get_meta_data_col('unique_id')
        last_ds = ts_loader.get_meta_data_col('last_ds') #TODO: ajustar of offset

        with t.no_grad():
            forecasts = []
            for batch in iter(ts_loader):
                insample_y  = self.to_tensor(x=batch['insample_y'])
                s_matrix    = self.to_tensor(x=batch['s_matrix'])
                idxs        = self.to_tensor(x=batch['idxs'], dtype=t.long)

                forecast = self.esrnn.predict(insample_y=insample_y, s_matrix=s_matrix, idxs=idxs)
                forecasts += [forecast.cpu().data.numpy()]
        forecasts = np.vstack(forecasts)

        # Predictions for panel
        Y_hat_panel = pd.DataFrame(columns=['unique_id', 'ds'])
        for i, unique_id in enumerate(unique_ids):
            Y_hat_id = pd.DataFrame([unique_id]*self.output_size, columns=["unique_id"])
            ds = pd.date_range(start=last_ds[i], periods=self.output_size+1, freq=frequency)
            Y_hat_id["ds"] = ds[1:]
            Y_hat_panel = Y_hat_panel.append(Y_hat_id, sort=False).reset_index(drop=True)

        Y_hat_panel['y_hat'] = forecasts.flatten()

        if X_test is not None:
            Y_hat_panel = X_test.merge(Y_hat_panel, on=['unique_id', 'ds'], how='left')
        
        return Y_hat_panel

    def save(self, model_dir=None, copy=None):
        """Auxiliary function to save ESRNN model"""
        if copy is not None:
                self.copy = copy

        if not model_dir:
            assert self.root_dir
            model_dir = self.get_dir_name()

        if not os.path.exists(model_dir):
            os.makedirs(model_dir)

        rnn_filepath = os.path.join(model_dir, "rnn.model")
        es_filepath = os.path.join(model_dir, "es.model")

        print('Saving model to:\n {}'.format(model_dir)+'\n')
        t.save({'model_state_dict': self.es.state_dict()}, es_filepath)
        t.save({'model_state_dict': self.rnn.state_dict()}, rnn_filepath)

    def load(self, model_dir=None, copy=None):
        """Auxiliary function to load ESRNN model"""
        if copy is not None:
            self.copy = copy

        if not model_dir:
            assert self.root_dir
            model_dir = self.get_dir_name()

        rnn_filepath = os.path.join(model_dir, "rnn.model")
        es_filepath = os.path.join(model_dir, "es.model")
        path = Path(es_filepath)

        if path.is_file():
            print('Loading model from:\n {}'.format(model_dir)+'\n')

            checkpoint = t.load(es_filepath, map_location=self.device)
            self.es.load_state_dict(checkpoint['model_state_dict'])
            self.es.to(self.device)

            checkpoint = t.load(rnn_filepath, map_location=self.device)
            self.rnn.load_state_dict(checkpoint['model_state_dict'])
            self.rnn.to(self.device)
        else:
            print('Model path {} does not exist'.format(path))


In [5]:
from nixtla.data.tsdataset import TimeSeriesDataset
from nixtla.data.tsloader_general import TimeSeriesLoader as TimeSeriesLoaderGeneral

In [6]:
from nixtla.data.datasets.tourism import Tourism, TourismInfo
group = TourismInfo.groups[0]
print("TourismInfo.groups[0]", group)
Y_df, _ = Tourism.load(directory='data', group=group)
tourism_dataset = TimeSeriesDataset(Y_df=Y_df, S_df=None, X_df=None, ts_train_mask=None)

TourismInfo.groups[0] Yearly
Processing dataframes ...
Creating ts tensor ...


In [7]:
train_loader = TimeSeriesLoaderGeneral(ts_dataset=tourism_dataset,
                                            model='esrnn',
                                            offset=4,
                                            window_sampling_limit=20*4, 
                                            input_size=2*4,
                                            output_size=4,
                                            idx_to_sample_freq=1,
                                            batch_size= 32,
                                            n_series_per_batch=32,
                                            is_train_loader=True)

In [8]:
esrnn = ESRNN(input_size=1*4,
              output_size=4,
              max_epochs=5,
              freq_of_test=-1,
              learning_rate=1e-3,
              lr_scheduler_step_size=9,
              lr_decay=0.9,
              per_series_lr_multip=1.0,
              gradient_eps=1e-8,
              gradient_clipping_threshold=20,
              rnn_weight_decay=0,
              noise_std=0.001,
              level_variability_penalty=200,
              testing_percentile=50,
              training_percentile=50,
              cell_type='LSTM',
              state_hsize=40,
              dilations=[[1, 2], [4, 8]],
              add_nl_layer=False,
              seasonality=[],
              random_seed=1,
              device='cpu')

In [9]:
esrnn.fit(train_ts_loader=train_loader, eval_epochs=1)

Infered frequency: A-DEC
=============== Training ESRNN  ===============

Epoch: 0, Time: 0.227, Insample loss: 5.07064
Epoch: 1, Time: 0.458, Insample loss: 4.94714
Epoch: 2, Time: 0.680, Insample loss: 5.24259
Epoch: 3, Time: 0.900, Insample loss: 4.84759
Epoch: 4, Time: 1.149, Insample loss: 4.86516


In [10]:
y_hat = esrnn.predict(ts_loader=train_loader)